This notebook is aimed to work out internal functions for MLP classification

In [1]:
# for all relative imports to work fine
import sys
sys.path.append("/home/sharfikeg/my_files/multinomial_diffusion")

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [9]:
# necessary imports
from model import get_model, get_model_id, add_model_args
from layers.layers import SegmentationUnet
from diffusion_utils.diffusion_multinomial import MultinomialDiffusion
from diffusion_utils.diffusion_multinomial import index_to_log_onehot,log_onehot_to_index

In [3]:
import numpy
import torch

/home/sharfikeg/anaconda3/envs/voxel/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# example input
array = numpy.load(
    "/home/sharfikeg/my_files/VoxelDiffusion/data/ShapeNet15k_voxels/dishwasher/5d17e90f512a3dc7df3a1b0d597ce76e.npy"
)
x = torch.tensor(array).unsqueeze(0).long().to('cuda')
time = torch.tensor([200]).to('cuda')

In [14]:
# models: UNet and Diffusion 
# UNet is used to gather activations
# Diffusion is used to get noisy samples
model = SegmentationUnet(
    num_classes=2,
    dim=32,
    num_steps=1000,
    dim_mults = (1, 8, 16, 32)
)

base_dist = MultinomialDiffusion(
    num_classes=2, shape=(1,32,32,32), denoise_fn=model, timesteps=1000
).to('cuda')

In [15]:
# example pretrained model loading
dict = torch.load("/home/sharfikeg/my_files/multinomial_diffusion/log/flow/shape_net_chairs/multinomial_diffusion/multistep/2023-03-15_16-21-31/check/checkpoint.pt")
base_dist.load_state_dict(dict['model'])

<All keys matched successfully>

In [9]:
# lines to 
# 1) convert input tensor to the necessary format
# 2) get noisy tensor
# 3) convert input tensor back to the format, suitable for UNet
log_x_start = index_to_log_onehot(x, num_classes=2)
log_x_t = base_dist.q_sample(log_x_start=log_x_start, t=time)
x_t = log_onehot_to_index(log_x_t)

In [ ]:
# usage of UNet
base_dist._denoise_fn(
    time=time,
    x=x_t
)

From here: work with Dima's code

In [5]:
import src.feature_extractor
device = 'cuda'

In [10]:
# feature extractor from Dima's
model_path = "/home/sharfikeg/my_files/multinomial_diffusion/log/flow/shape_net_chairs/multinomial_diffusion/multistep/2023-03-15_16-21-31/check/checkpoint.pt"
exctractor = src.feature_extractor.create_feature_extractor(
    model_type='ddpm',
    model_path=model_path,
    input_activations=False,
    num_classes=2,
    num_steps=1000,
    dim=32,
    shape=(1,32,32,32),
    dim_mults=(1,8,16,32),
    steps=[100],
    blocks=[0,1,2],
    device=device
)

Creating DDPM Feature Extractor...
Pretrained model is successfully loaded from /home/sharfikeg/my_files/multinomial_diffusion/log/flow/shape_net_chairs/multinomial_diffusion/multistep/2023-03-15_16-21-31/check/checkpoint.pt
0
1
2
0


In [11]:
src.feature_extractor.collect_features({"shape":(1,32,32,32), "upsample_mode":"trilinear"},exctractor(x, None)).shape

torch.Size([1824, 32, 32, 32])

In [15]:
exctractor(x, None)[0].shape

torch.Size([1, 512, 4, 4, 4])

In [5]:
from torch.utils.data import Dataset
import blobfile as bf

In [24]:
def _list_voxel_files_recursively(data_dir):
    results = []
    for entry in sorted(bf.listdir(data_dir)):
        full_path = bf.join(data_dir, entry)
        ext = entry.split(".")[-1]
        if "." in entry and ext.lower() in ["npy"]:
            results.append(full_path)
        elif bf.isdir(full_path):
            results.extend(_list_voxel_files_recursively(full_path))
    return results

In [25]:
class ImageLabelDataset(Dataset):
    ''' 
    :param data_dir: path to a folder with images and their annotations. 
                     Annotations are supposed to be in *.npy format.
    :param resolution: image and mask output resolution.
    :param num_images: restrict a number of images in the dataset.
    :param transform: image transforms.
    '''
    def __init__(
        self,
        data_dir: str,
        resolution: int,
        num_images= -1,
        transform=None,
    ):
        super().__init__()
        self.resolution = resolution
        self.transform = transform
        self.image_paths = _list_voxel_files_recursively(data_dir)
        self.image_paths = sorted(self.image_paths)

        if num_images > 0:
            print(f"Take first {num_images} images...")
            self.image_paths = self.image_paths[:num_images]

        

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load an image
        image_path = self.image_paths[idx]
        object_and_mask = numpy.load(image_path)
        

        # assert pil_image.size[0] == pil_image.size[1], \
        #        f"Only square images are supported: ({pil_image.size[0]}, {pil_image.size[1]})"

        tensor_object_and_mask = self.transform(object_and_mask)
        tensor_object = tensor_object_and_mask[0].unsqueeze(0)
        # Load a corresponding mask and resize it to (self.resolution, self.resolution)
        # label_path = self.label_paths[idx]
        # label = np.load(label_path).astype('uint8')
        # label = cv2.resize(
        #     label, (self.resolution, self.resolution), interpolation=cv2.INTER_NEAREST
        # )
        tensor_label = tensor_object_and_mask[1:,...]
        return tensor_object, tensor_label

In [26]:
d = ImageLabelDataset(
    "/home/sharfikeg/my_files/VoxelDiffusion/data/ShapeNet_annotated_voxels/chair",
    32,
    10,
    lambda array: (torch.FloatTensor(array)).long()
)

Take first 10 images...


In [27]:
for data in d:
    print(data[0].shape[:-1])
    break

torch.Size([1, 32, 32])


In [28]:
# prepare data for MLPs training
def prepare_data(args):
    feature_extractor = src.feature_extractor.create_feature_extractor(**args)
    
    print(f"Preparing the train set for ...")
    dataset = ImageLabelDataset(
        data_dir=args['training_path'],
        resolution=args['image_size'],
        num_images=args['training_number'],
        transform=lambda array: (torch.FloatTensor(array)).long()
    )
    X = torch.zeros((len(dataset),args['num_of_features'] , *args['shape'][1:]), dtype=torch.uint8)
    y = torch.zeros((len(dataset), *args['shape'][1:]), dtype=torch.uint8)

    if 'share_noise' in args and args['share_noise']:
        rnd_gen = torch.Generator(device=args['device']).manual_seed(args['seed'])
        noise = torch.randn(1, 3, args['image_size'], args['image_size'], 
                            generator=rnd_gen, device=args['device'])
    else:
        noise = None 

    for row, (img, label) in enumerate(tqdm(dataset)):
        img = img[None].to(args['device'])
        features = feature_extractor(img, noise=noise)
        X[row] = src.feature_extractor.collect_features(args, features).cpu()
        
        # for target in range(args['number_class']):
        #     if target == args['ignore_label']: continue
        #     if 0 < (label == target).sum() < 20:
        #         print(f'Delete small annotation from image {dataset.image_paths[row]} | label {target}')
        #         label[label == target] = args['ignore_label']
        y[row] = label.argmax(0)
    
    d = X.shape[1]
    print(f'Total dimension {d}')
    X = X.permute(1,0,2,3,4).reshape(d, -1).permute(1, 0)
    y = y.flatten()
    return X, y

In [6]:
from tqdm import tqdm

In [ ]:
model_path = "/home/sharfikeg/my_files/multinomial_diffusion/log/flow/shape_net_chairs/multinomial_diffusion/multistep/2023-03-09_10-47-15/check/checkpoint.pt"
exp_path = "/home/sharfikeg/my_files/multinomial_diffusion/segmentation_diffusion/exp_dir"
args = {
    'model_type':'ddpm',
    'model_path':model_path,
    'input_activations':False,
    'num_classes':2,
    'num_steps':1000,
    'dim':32,
    'shape':(1,32,32,32),
    'num_of_features':224*2,
    'steps':[250, 300],
    'blocks':[0,1,2],
    'training_path':"/home/sharfikeg/my_files/VoxelDiffusion/data/ShapeNet_annotated_voxels/chair",
    'image_size':32,
    'training_number':3010,
    'share_noise':False,
    'device': 'cuda',
    'number_class': 5,
    'upsample_mode':"trilinear",
    'start_model_num': 0,
    'model_num': 1,
    'exp_dir': exp_path,
    'batch_size': 16,
    }


X,y = prepare_data(
    args
)

In [7]:
from train_mlp import train

In [ ]:
model_path = "/home/sharfikeg/my_files/multinomial_diffusion/log/flow/shape_net_chairs/multinomial_diffusion/multistep/2023-03-09_10-47-15/check/checkpoint.pt"
exp_path = "/home/sharfikeg/my_files/multinomial_diffusion/segmentation_diffusion/exp_dir"
args = {
    'model_type':'ddpm',
    'model_path':model_path,
    'input_activations':False,
    'num_classes':2,
    'num_steps':1000,
    'dim':32,
    'shape':(1,32,32,32),
    'num_of_features':224*2,
    'steps':[250, 300],
    'blocks':[0,1,2],
    'training_path':"/home/sharfikeg/my_files/VoxelDiffusion/data/ShapeNet_annotated_voxels/chair",
    'image_size':32,
    'training_number':10,
    'share_noise':False,
    'device': 'cuda',
    'number_class': 5,
    'upsample_mode':"trilinear",
    'start_model_num': 0,
    'model_num': 1,
    'ignore_label': 1,
    'exp_dir': exp_path,
    'batch_size': 16,
    }

train(args)